# **Import Library**

In [6]:
!pip install Sastrawi

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

# **Data Exploration**

In [8]:
# Persiapan Dataset
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/DatasetColabNotebooks/data_clean_analytics_femaledaily.csv')
df.head(5)

Mounted at /content/drive


,category,subcategory,product_name,price,description,user_name,star_rating,is_recommend,review_date,user_review,product_brand
0,Treatment,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,wulanraahma,5.0,Yes,07 Oct 2021,Teksturnya runny gel dan ada edible gold nya. ...,AAGS Skincare
1,Treatment,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,sofiahdynt,5.0,Yes,03 Sep 2021,"Teksturnya gel like, agak slimey, dan mudah di...",AAGS Skincare
2,Moisturizer,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",wulanraahma,5.0,Yes,07 Oct 2021,Untuk hasilnya yang aku rasain itu ngelembapin...,AAGS Skincare
3,Moisturizer,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",sofiahdynt,4.0,Yes,03 Sep 2021,Teksturnya gel cream berwarna kuning muda deng...,AAGS Skincare
4,Moisturizer,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",popy_mercylin,5.0,Yes,19 Feb 2023,night cream ini salah satu cream yang jadi fav...,Adera


In [9]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29003 entries, 0 to 29002
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   category       29003 non-null  object 
 1   subcategory    29003 non-null  object 
 2   product_name   29003 non-null  object 
 3   price          29003 non-null  object 
 4   description    29003 non-null  object 
 5   user_name      29003 non-null  object 
 6   star_rating    29003 non-null  float64
 7   is_recommend   29003 non-null  object 
 8   review_date    29003 non-null  object 
 9   user_review    29003 non-null  object 
 10  product_brand  29003 non-null  object 
dtypes: float64(1), object(10)
memory usage: 2.4+ MB


In [10]:
# Memilih Kolom
df = df[['product_name','user_review','product_brand','category', 'subcategory', 'description', 'star_rating', 'is_recommend']]
df.head(5)

,product_name,user_review,product_brand,category,subcategory,description,star_rating,is_recommend
0,All in One Miracle Serum,Teksturnya runny gel dan ada edible gold nya. ...,AAGS Skincare,Treatment,Serum & Essence,All in one mircale serum ini mengandung niacin...,5.0,Yes
1,All in One Miracle Serum,"Teksturnya gel like, agak slimey, dan mudah di...",AAGS Skincare,Treatment,Serum & Essence,All in one mircale serum ini mengandung niacin...,5.0,Yes
2,Glowing Night Cream,Untuk hasilnya yang aku rasain itu ngelembapin...,AAGS Skincare,Moisturizer,Cream,"Night cream ini mengandung cucumber extract, n...",5.0,Yes
3,Glowing Night Cream,Teksturnya gel cream berwarna kuning muda deng...,AAGS Skincare,Moisturizer,Cream,"Night cream ini mengandung cucumber extract, n...",4.0,Yes
4,Glowing Night Cream,night cream ini salah satu cream yang jadi fav...,Adera,Moisturizer,Cream,"Night cream ini mengandung cucumber extract, n...",5.0,Yes


In [11]:
# Menggabungkan semua review dari produk yang sama
combined_reviews = df.groupby('product_name')['user_review'].apply(lambda x: ' '.join(x)).reset_index()

# Menyiapkan dataframe hasil penggabungan review
combined_df = pd.DataFrame(combined_reviews, columns=['product_name', 'user_review'])

# Merge dengan dataset asli untuk mendapatkan kolom-kolom lain
combined_df = pd.merge(combined_df, df[['product_name','product_brand', 'category', 'subcategory','description','star_rating','is_recommend']], on='product_name', how='left')

# Menghapus produk dengan nilai duplikat berdasarkan 'product_name'
combined_df = combined_df.drop_duplicates(subset=['product_name'])

# Membuat combined_df menjadi df utama
df = combined_df

# Menampilkan hasil penggabungan review dengan kolom tambahan
df.head()

,product_name,user_review,product_brand,category,subcategory,description,star_rating,is_recommend
0,01 Radiance Skin Exfoliate Serum/Peeling Serum,Bangga sama produk buatan Sidoarjo satu ini. U...,Lysca,Treatment,Serum & Essence,"Lysca Radiance Skin AHA 3%, BHA 1%, PHA 2% + W...",5.0,Yes
5,02 Bounce Skin Hydrate Serum,kemaren lagi nyari serum hydration yg murce te...,Lysca,Treatment,Serum & Essence,Lysca Bounce Skin Hyaluronic Acid 1%* + Green ...,5.0,Yes
14,03 Glow Skin Brightening Serum,"Brand ini ternyata dr Sidoarjo, kota tempat ti...",Lysca,Treatment,Serum & Essence,Lysca Glow Skin Niacinamide 10% + Korean Galac...,5.0,Yes
20,04 Blemish Skin Anti-Acne Serum,setelah drama beli cleanser palsu krn ga beli ...,Lysca,Treatment,Serum & Essence,Lysca Blemish Skin Succinic Acid/BHA 2% + CICA...,5.0,Yes
25,1% Actosome Retinol Serum,Setelah menimbang nimbang akhirnya cobain ini ...,Re.Set the Skin,Treatment,Serum & Essence,Re.Set the Skin 1% Actosome Retinol Serum meru...,5.0,Yes


In [12]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3742 entries, 0 to 29002
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_name   3742 non-null   object 
 1   user_review    3742 non-null   object 
 2   product_brand  3742 non-null   object 
 3   category       3742 non-null   object 
 4   subcategory    3742 non-null   object 
 5   description    3742 non-null   object 
 6   star_rating    3742 non-null   float64
 7   is_recommend   3742 non-null   object 
dtypes: float64(1), object(7)
memory usage: 263.1+ KB


# **Data Preprocessing**

In [18]:
nltk.download('stopwords')
nltk.download('punkt')
# Define text preprocessing methods
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words('indonesian'))

def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F000-\U0001F6FF"  # emoticons & transportasi
        u"\U0001F780-\U0001F7FF"  # simbol, tanda & bendera
        u"\U0001F800-\U0001F8FF"  # emoji surat / katakana
        u"\U0001F900-\U0001F9FF"  # emoji berbagai jenis
        u"\U00002600-\U000026FF"  # simbol matahari & bulan
        u"\U00002700-\U000027BF"  # simbol koin, alat musik, dll.
        u"\U0001F300-\U0001F5FF"  # simbol & markah
        u"\U0001F680-\U0001F6FF"  # transportasi & simbol tempat
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def preprocess_text(text):
   # Remove emoji
    text = remove_emoji(text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Case folding
    text = text.lower()
    # Tokenization
    words = nltk.word_tokenize(text)
    # Filtering
    filtered_words = [word for word in words if re.match(r'[a-zA-Z]+', word) and not word in stop_words]
    # Stemming
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    # Joining the stemmed words back into a single string
    preprocessed_text = " ".join(stemmed_words)
    return preprocessed_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
# Apply the text preprocessing function to the "review" column
df["description_processed"] = df["description"].apply(preprocess_text)

# Apply the text preprocessing function to the "review" column
df["review_processed"] = df["user_review"].apply(preprocess_text)

In [21]:
df

,product_name,user_review,product_brand,category,subcategory,description,star_rating,is_recommend,description_processed,review_processed
0,01 Radiance Skin Exfoliate Serum/Peeling Serum,Bangga sama produk buatan Sidoarjo satu ini. U...,Lysca,Treatment,Serum & Essence,"Lysca Radiance Skin AHA 3%, BHA 1%, PHA 2% + W...",5.0,Yes,lysca radiance skin aha bha pha witch hazel ex...,bangga produk buat sidoarjo harga yg oke bange...
5,02 Bounce Skin Hydrate Serum,kemaren lagi nyari serum hydration yg murce te...,Lysca,Treatment,Serum & Essence,Lysca Bounce Skin Hyaluronic Acid 1%* + Green ...,5.0,Yes,lysca bounce skin hyaluronic acid green caviar...,kemaren nyari serum hydration yg murce jatuh l...
14,03 Glow Skin Brightening Serum,"Brand ini ternyata dr Sidoarjo, kota tempat ti...",Lysca,Treatment,Serum & Essence,Lysca Glow Skin Niacinamide 10% + Korean Galac...,5.0,Yes,lysca glow skin niacinamide korean galactomyce...,brand dr sidoarjo kota tinggal kemas warna cai...
20,04 Blemish Skin Anti-Acne Serum,setelah drama beli cleanser palsu krn ga beli ...,Lysca,Treatment,Serum & Essence,Lysca Blemish Skin Succinic Acid/BHA 2% + CICA...,5.0,Yes,lysca blemish skin succinic acid bha cica anti...,drama beli cleanser palsu krn ga beli official...
25,1% Actosome Retinol Serum,Setelah menimbang nimbang akhirnya cobain ini ...,Re.Set the Skin,Treatment,Serum & Essence,Re.Set the Skin 1% Actosome Retinol Serum meru...,5.0,Yes,re set the skin actosome retinol serum serum r...,timbang nimbang cobain duluan packagingnya luc...
...,...,...,...,...,...,...,...,...,...,...
28982,suncare with AHA,waktuuuuu pertama kali coba ternyata produk ny...,Theraskin Skincare,Moisturizer,Sun Protection,In general Suncare AHA can be used for all ski...,5.0,Yes,in general suncare aha can be used for all ski...,waktuuuuu kali coba produk nya cocok bgtttt ga...
28985,valenno facial wash with papaya extract,Waktu dulu banget pernah pakai ini. Dulu sempa...,Valenno,Cleanser,Facial Wash,There's no description for this product.,4.0,Yes,there no description for this product,banget pakai repurchase kali face washnya buli...
28990,vylea,Kelewat murah sih 25k an isinya 7 sheet mask. ...,Vivelle,Mask,Mask Sheet,"mengandung tea tree oil sebagai anti bacteri, ...",2.0,No,kandung tea tree oil anti bacteri sodium hyalu...,lewat murah sih k an isi sheet mask kertas teb...
28992,whitening soap,Klaimnya aman buat tubuh dan wajah. Saat kehab...,Papaya,Cleanser,Facial Wash,facial foam soap,1.0,No,facial foam soap,klaim aman tubuh wajah habis facial wash habis...


In [20]:
df.to_csv('data_final_clean_femaledaily.csv', index=False)

In [45]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/DatasetColabNotebooks/data_final_clean_femaledaily.csv')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,product_name,user_review,product_brand,category,subcategory,description,star_rating,is_recommend,description_processed,review_processed
0,01 Radiance Skin Exfoliate Serum/Peeling Serum,Bangga sama produk buatan Sidoarjo satu ini. U...,Lysca,Treatment,Serum & Essence,"Lysca Radiance Skin AHA 3%, BHA 1%, PHA 2% + W...",5.0,Yes,lysca radiance skin aha bha pha witch hazel ex...,bangga produk buat sidoarjo harga yg oke bange...
1,02 Bounce Skin Hydrate Serum,kemaren lagi nyari serum hydration yg murce te...,Lysca,Treatment,Serum & Essence,Lysca Bounce Skin Hyaluronic Acid 1%* + Green ...,5.0,Yes,lysca bounce skin hyaluronic acid green caviar...,kemaren nyari serum hydration yg murce jatuh l...
2,03 Glow Skin Brightening Serum,"Brand ini ternyata dr Sidoarjo, kota tempat ti...",Lysca,Treatment,Serum & Essence,Lysca Glow Skin Niacinamide 10% + Korean Galac...,5.0,Yes,lysca glow skin niacinamide korean galactomyce...,brand dr sidoarjo kota tinggal kemas warna cai...
3,04 Blemish Skin Anti-Acne Serum,setelah drama beli cleanser palsu krn ga beli ...,Lysca,Treatment,Serum & Essence,Lysca Blemish Skin Succinic Acid/BHA 2% + CICA...,5.0,Yes,lysca blemish skin succinic acid bha cica anti...,drama beli cleanser palsu krn ga beli official...
4,1% Actosome Retinol Serum,Setelah menimbang nimbang akhirnya cobain ini ...,Re.Set the Skin,Treatment,Serum & Essence,Re.Set the Skin 1% Actosome Retinol Serum meru...,5.0,Yes,re set the skin actosome retinol serum serum r...,timbang nimbang cobain duluan packagingnya luc...
...,...,...,...,...,...,...,...,...,...,...
3737,suncare with AHA,waktuuuuu pertama kali coba ternyata produk ny...,Theraskin Skincare,Moisturizer,Sun Protection,In general Suncare AHA can be used for all ski...,5.0,Yes,in general suncare aha can be used for all ski...,waktuuuuu kali coba produk nya cocok bgtttt ga...
3738,valenno facial wash with papaya extract,Waktu dulu banget pernah pakai ini. Dulu sempa...,Valenno,Cleanser,Facial Wash,There's no description for this product.,4.0,Yes,there no description for this product,banget pakai repurchase kali face washnya buli...
3739,vylea,Kelewat murah sih 25k an isinya 7 sheet mask. ...,Vivelle,Mask,Mask Sheet,"mengandung tea tree oil sebagai anti bacteri, ...",2.0,No,kandung tea tree oil anti bacteri sodium hyalu...,lewat murah sih k an isi sheet mask kertas teb...
3740,whitening soap,Klaimnya aman buat tubuh dan wajah. Saat kehab...,Papaya,Cleanser,Facial Wash,facial foam soap,1.0,No,facial foam soap,klaim aman tubuh wajah habis facial wash habis...


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3742 entries, 0 to 3741
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_name           3742 non-null   object 
 1   user_review            3742 non-null   object 
 2   product_brand          3742 non-null   object 
 3   category               3742 non-null   object 
 4   subcategory            3742 non-null   object 
 5   description            3742 non-null   object 
 6   star_rating            3742 non-null   float64
 7   is_recommend           3742 non-null   object 
 8   description_processed  3741 non-null   object 
 9   review_processed       3742 non-null   object 
dtypes: float64(1), object(9)
memory usage: 292.5+ KB


In [47]:
df = df.dropna()

In [48]:
# reset index
df = df.reset_index(drop=True)
df

,product_name,user_review,product_brand,category,subcategory,description,star_rating,is_recommend,description_processed,review_processed
0,01 Radiance Skin Exfoliate Serum/Peeling Serum,Bangga sama produk buatan Sidoarjo satu ini. U...,Lysca,Treatment,Serum & Essence,"Lysca Radiance Skin AHA 3%, BHA 1%, PHA 2% + W...",5.0,Yes,lysca radiance skin aha bha pha witch hazel ex...,bangga produk buat sidoarjo harga yg oke bange...
1,02 Bounce Skin Hydrate Serum,kemaren lagi nyari serum hydration yg murce te...,Lysca,Treatment,Serum & Essence,Lysca Bounce Skin Hyaluronic Acid 1%* + Green ...,5.0,Yes,lysca bounce skin hyaluronic acid green caviar...,kemaren nyari serum hydration yg murce jatuh l...
2,03 Glow Skin Brightening Serum,"Brand ini ternyata dr Sidoarjo, kota tempat ti...",Lysca,Treatment,Serum & Essence,Lysca Glow Skin Niacinamide 10% + Korean Galac...,5.0,Yes,lysca glow skin niacinamide korean galactomyce...,brand dr sidoarjo kota tinggal kemas warna cai...
3,04 Blemish Skin Anti-Acne Serum,setelah drama beli cleanser palsu krn ga beli ...,Lysca,Treatment,Serum & Essence,Lysca Blemish Skin Succinic Acid/BHA 2% + CICA...,5.0,Yes,lysca blemish skin succinic acid bha cica anti...,drama beli cleanser palsu krn ga beli official...
4,1% Actosome Retinol Serum,Setelah menimbang nimbang akhirnya cobain ini ...,Re.Set the Skin,Treatment,Serum & Essence,Re.Set the Skin 1% Actosome Retinol Serum meru...,5.0,Yes,re set the skin actosome retinol serum serum r...,timbang nimbang cobain duluan packagingnya luc...
...,...,...,...,...,...,...,...,...,...,...
3736,suncare with AHA,waktuuuuu pertama kali coba ternyata produk ny...,Theraskin Skincare,Moisturizer,Sun Protection,In general Suncare AHA can be used for all ski...,5.0,Yes,in general suncare aha can be used for all ski...,waktuuuuu kali coba produk nya cocok bgtttt ga...
3737,valenno facial wash with papaya extract,Waktu dulu banget pernah pakai ini. Dulu sempa...,Valenno,Cleanser,Facial Wash,There's no description for this product.,4.0,Yes,there no description for this product,banget pakai repurchase kali face washnya buli...
3738,vylea,Kelewat murah sih 25k an isinya 7 sheet mask. ...,Vivelle,Mask,Mask Sheet,"mengandung tea tree oil sebagai anti bacteri, ...",2.0,No,kandung tea tree oil anti bacteri sodium hyalu...,lewat murah sih k an isi sheet mask kertas teb...
3739,whitening soap,Klaimnya aman buat tubuh dan wajah. Saat kehab...,Papaya,Cleanser,Facial Wash,facial foam soap,1.0,No,facial foam soap,klaim aman tubuh wajah habis facial wash habis...


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3741 entries, 0 to 3740
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_name           3741 non-null   object 
 1   user_review            3741 non-null   object 
 2   product_brand          3741 non-null   object 
 3   category               3741 non-null   object 
 4   subcategory            3741 non-null   object 
 5   description            3741 non-null   object 
 6   star_rating            3741 non-null   float64
 7   is_recommend           3741 non-null   object 
 8   description_processed  3741 non-null   object 
 9   review_processed       3741 non-null   object 
dtypes: float64(1), object(9)
memory usage: 292.4+ KB


# **TF-IDF & Cosine Similarity**

In [50]:
# Representasi Fitur
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['description_processed'] + ' ' + df['review_processed'])

In [51]:
print(tfidf_matrix)

  (0, 29077)	0.06026530059235786
  (0, 2508)	0.03498146699612221
  (0, 7103)	0.05153383763540728
  (0, 20083)	0.038009914129006034
  (0, 17299)	0.01981322937950068
  (0, 9185)	0.05597460010772417
  (0, 3611)	0.045492916515285586
  (0, 6705)	0.037481024090907505
  (0, 7002)	0.020622007102510186
  (0, 15209)	0.019916068803439227
  (0, 14177)	0.05479556798475917
  (0, 4169)	0.0797076315942865
  (0, 10721)	0.060040779022771065
  (0, 14038)	0.03576370601748369
  (0, 2958)	0.035126141738336426
  (0, 20441)	0.0868309902996854
  (0, 3546)	0.016890843536606952
  (0, 5183)	0.034692585923027604
  (0, 5179)	0.04483459095415903
  (0, 3683)	0.027544859190865897
  (0, 10602)	0.027529321151352787
  (0, 34283)	0.016430576521950627
  (0, 22983)	0.035178226272518334
  (0, 17149)	0.06337685697464117
  (0, 2924)	0.022001318112667825
  :	:
  (3740, 14508)	0.020953901701702145
  (3740, 5913)	0.042273254823608185
  (3740, 23356)	0.0290032422543548
  (3740, 23414)	0.018825103483086542
  (3740, 2396)	0.07148759

## Contoh Implementasi Penerapan TF-IDF

In [52]:
# Kata-kata yang ingin dihitung TF-nya
target_words = ["bersih", "cocok", "facial", "jerawat", "kering", "kulit"]

# Mengambil indeks kolom yang sesuai dengan kata-kata target
target_word_indices = [tfidf.vocabulary_.get(word) for word in target_words]

# Menghitung TF kata-kata target
tf_scores = tfidf_matrix[:, target_word_indices].toarray()

# Membuat DataFrame untuk menyimpan hasil TF
tf_df = pd.DataFrame(tf_scores, columns=target_words)

# Menambahkan kolom 'Dokumen' ke DataFrame
tf_df['Dokumen'] = range(1, len(df)+1)

# Mengatur 'Dokumen' sebagai indeks
tf_df.set_index('Dokumen', inplace=True)

# Menampilkan DataFrame
display(tf_df)

,bersih,cocok,facial,jerawat,kering,kulit
Dokumen,,,,,,
1,0.000000,0.060967,0.000000,0.030220,0.000000,0.082458
2,0.000000,0.037162,0.000000,0.000000,0.038563,0.191476
3,0.000000,0.017042,0.000000,0.084475,0.017684,0.105370
4,0.042822,0.033999,0.000000,0.269643,0.000000,0.065692
5,0.015069,0.071787,0.000000,0.047444,0.012415,0.110963
...,...,...,...,...,...,...
3737,0.000000,0.012994,0.000000,0.012881,0.000000,0.000000
3738,0.025261,0.120339,0.038960,0.019883,0.041625,0.046503
3739,0.000000,0.035503,0.000000,0.000000,0.000000,0.027439


In [53]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Inisialisasi CountVectorizer
vectorizer = CountVectorizer(vocabulary=target_words)

# Menghitung TF kata-kata target
tf_matrix = vectorizer.transform(df['description_processed'] + ' ' + df['review_processed'])

# Membuat DataFrame untuk menyimpan hasil TF
tf_df = pd.DataFrame(tf_matrix.toarray(), columns=target_words)

# Menambahkan kolom 'Dokumen' ke DataFrame
tf_df['Dokumen'] = range(1, len(df)+1)

# Mengatur 'Dokumen' sebagai indeks
tf_df.set_index('Dokumen', inplace=True)

# Menampilkan DataFrame
display(tf_df)

,bersih,cocok,facial,jerawat,kering,kulit
Dokumen,,,,,,
1,0,4,0,2,0,7
2,0,3,0,0,3,20
3,0,1,0,5,1,8
4,2,2,0,16,0,5
5,1,6,0,4,1,12
...,...,...,...,...,...,...
3737,0,1,0,1,0,0
3738,1,6,1,1,2,3
3739,0,1,0,0,0,1


## Penerapan Cosine Similarity

In [54]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(tfidf_matrix)

In [55]:
print(cosine_similarities)

[[1.         0.28274304 0.32799082 ... 0.07630981 0.08435088 0.13161318]
 [0.28274304 1.         0.48602841 ... 0.10768947 0.13182421 0.10817347]
 [0.32799082 0.48602841 1.         ... 0.09501971 0.14726994 0.12010031]
 ...
 [0.07630981 0.10768947 0.09501971 ... 1.         0.10944869 0.04149783]
 [0.08435088 0.13182421 0.14726994 ... 0.10944869 1.         0.1052688 ]
 [0.13161318 0.10817347 0.12010031 ... 0.04149783 0.1052688  1.        ]]


In [56]:
cosine_df = pd.DataFrame(cosine_similarities)
cosine_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3731,3732,3733,3734,3735,3736,3737,3738,3739,3740
0,1.000000,0.282743,0.327991,0.347895,0.166133,0.167966,0.113143,0.176707,0.166647,0.177214,...,0.062669,0.072924,0.077398,0.020724,0.070913,0.100712,0.054057,0.076310,0.084351,0.131613
1,0.282743,1.000000,0.486028,0.337321,0.165758,0.264580,0.166712,0.241913,0.223968,0.301536,...,0.070868,0.122522,0.113893,0.031947,0.114642,0.053006,0.077929,0.107689,0.131824,0.108173
2,0.327991,0.486028,1.000000,0.385378,0.192865,0.254534,0.168889,0.261104,0.237975,0.226326,...,0.054400,0.124471,0.104783,0.053367,0.098633,0.041354,0.060758,0.095020,0.147270,0.120100
3,0.347895,0.337321,0.385378,1.000000,0.172448,0.223772,0.140136,0.271559,0.211871,0.262295,...,0.072832,0.105117,0.146363,0.025729,0.070219,0.037289,0.070896,0.071320,0.112771,0.125876
4,0.166133,0.165758,0.192865,0.172448,1.000000,0.404192,0.310233,0.420429,0.156450,0.201422,...,0.078113,0.138383,0.115378,0.048245,0.120194,0.074755,0.105050,0.069294,0.155079,0.112044


# Recommendation Analysis

## Input Product for Recommendation

In [63]:
similarities = {}

for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the product.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    similarities[df['product_name'].iloc[i]] = [(cosine_similarities[i][x], df['product_name'][x], df['product_brand'][x]) for x in similar_indices][1:]

In [64]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, skincare, recom_skincare):
        rec_items = len(recom_skincare)

        print(f'The {rec_items} recommended product for {skincare} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_skincare[i][1]} brand {recom_skincare[i][2]} with {round(recom_skincare[i][0], 3)} similarity score")
            print("--------------------")

    def recommend(self, recommendation):
        # Get product to find recommendations for
        skincare = recommendation['product_name']
        # Get number of product to recommend
        sum_skincare = recommendation['sum_skincare']
        # Get the number of product most similars from matrix similarities
        recom_skincare = self.matrix_similar[skincare][:sum_skincare]
        # print each item
        self._print_message(skincare=skincare, recom_skincare=recom_skincare)

In [65]:
recommedations = ContentBasedRecommender(similarities)

In [66]:
recommendation = {
    "product_name": 'vylea',
    "sum_skincare": 5
}

In [67]:
recommedations.recommend(recommendation)

The 5 recommended product for vylea are:
Number 1:
Glow Recipe Juicy Sheet Mask brand Citra with 0.282 similarity score
--------------------
Number 2:
Facial Mask brand Guardian with 0.252 similarity score
--------------------
Number 3:
Spa system tencel mask brand Vivelle with 0.25 similarity score
--------------------
Number 4:
White-Aqua Serum Sheet Mask brand PIXY with 0.249 similarity score
--------------------
Number 5:
Hydrating Nourishing Sheet Mask brand Airinderm Aesthetic with 0.238 similarity score
--------------------


## If Input Multiple Product

In [68]:
similarities = {}

for i in range(len(cosine_similarities)):
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    similarities[df['product_name'].iloc[i]] = [(cosine_similarities[i][x], df['product_name'][x], df['product_brand'][x]) for x in similar_indices][1:]

In [69]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, skincare, recom_skincare):
        rec_items = len(recom_skincare)

        print(f'The {rec_items} recommended products for {skincare} are:')
        for i, product in enumerate(recom_skincare, start=1):
            print(f"Number {i}:")
            print(f"{product[1]} brand {product[2]} with {round(product[0], 3)} similarity score")
            print("--------------------")

    def recommend(self, recommendation):
        # Get products to find recommendations for
        skincare_list = recommendation['product_names']
        # Get number of products to recommend
        sum_skincare = recommendation['sum_skincare']

        all_recom_skincare = []
        for skincare in skincare_list:
            if skincare in self.matrix_similar:
                recom_skincare = self.matrix_similar[skincare][:sum_skincare]
                all_recom_skincare.extend(recom_skincare)

        all_recom_skincare = sorted(all_recom_skincare, key=lambda x: x[0], reverse=True)

        print(f"The recommended products based on multiple inputs are:")
        for i, product in enumerate(all_recom_skincare[:sum_skincare], start=1):
            print(f"Number {i}:")
            print(f"{product[1]} brand {product[2]} with {round(product[0], 3)} similarity score")
            print("--------------------")

In [70]:
recommedations = ContentBasedRecommender(similarities)

In [71]:
recommendation = {
    "product_names": ['wardah face wash', 'valenno facial wash', 'vylea'],
    "sum_skincare": 5
}

In [72]:
recommedations.recommend(recommendation)

The recommended products based on multiple inputs are:
Number 1:
Glow Recipe Juicy Sheet Mask brand Citra with 0.282 similarity score
--------------------
Number 2:
Facial Mask brand Guardian with 0.252 similarity score
--------------------
Number 3:
Spa system tencel mask brand Vivelle with 0.25 similarity score
--------------------
Number 4:
White-Aqua Serum Sheet Mask brand PIXY with 0.249 similarity score
--------------------
Number 5:
Hydrating Nourishing Sheet Mask brand Airinderm Aesthetic with 0.238 similarity score
--------------------
